In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model, Input

# ------------------------------
# 1. Generate dummy dataset (regression + classification)
# ------------------------------
# Regression: y = 3x + noise
X_reg = np.random.rand(1000, 5)
y_reg = 3 * X_reg[:, 0] + 2 * X_reg[:, 1] + np.random.randn(1000)

# Classification: binary labels from same X
y_clf = (X_reg[:, 0] + X_reg[:, 1] > 1).astype(int)

# ------------------------------
# 2. Functional API Model
# ------------------------------
inputs = Input(shape=(5,), name="input_layer")

# Hidden layers
x = layers.Dense(64, activation="relu")(inputs)
x = layers.Dropout(0.3)(x)   # prevent overfitting
x = layers.Dense(32, activation="relu")(x)

# Two outputs (regression + classification)
reg_output = layers.Dense(1, name="regression_output")(x)
clf_output = layers.Dense(1, activation="sigmoid", name="classification_output")(x)

# Create model
model = Model(inputs=inputs, outputs=[reg_output, clf_output], name="multi_task_model")

# ------------------------------
# 3. Compile
# ------------------------------
model.compile(
    optimizer="adam",
    loss={"regression_output": "mse", "classification_output": "binary_crossentropy"},
    metrics={"regression_output": "mae", "classification_output": "accuracy"},
)

# ------------------------------
# 4. Train
# ------------------------------
history = model.fit(
    X_reg, {"regression_output": y_reg, "classification_output": y_clf},
    validation_split=0.2,
    epochs=10,
    batch_size=32
)

# ------------------------------
# 5. Save and Load
# ------------------------------
model.save("multi_task_model.keras")
reloaded_model = keras.models.load_model("multi_task_model.keras")
print("Model reloaded successfully!")

# ------------------------------
# 6. Hyperparameter Tuning Example (simple loop)
# ------------------------------
def build_model(units=64, dropout=0.3):
    inputs = Input(shape=(5,))
    x = layers.Dense(units, activation="relu")(inputs)
    x = layers.Dropout(dropout)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    model = Model(inputs, outputs)
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model

for units in [32, 64, 128]:
    print(f"\nTraining with {units} units")
    temp_model = build_model(units=units)
    temp_model.fit(X_reg, y_clf, epochs=5, batch_size=32, verbose=0)


Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - classification_output_accuracy: 0.4876 - classification_output_loss: 0.8162 - loss: 5.8284 - regression_output_loss: 5.0122 - regression_output_mae: 1.8739 - val_classification_output_accuracy: 0.4950 - val_classification_output_loss: 0.8363 - val_loss: 3.2357 - val_regression_output_loss: 2.2968 - val_regression_output_mae: 1.2708
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - classification_output_accuracy: 0.5011 - classification_output_loss: 0.8713 - loss: 2.9050 - regression_output_loss: 2.0337 - regression_output_mae: 1.1324 - val_classification_output_accuracy: 0.4950 - val_classification_output_loss: 0.7899 - val_loss: 2.1702 - val_regression_output_loss: 1.3679 - val_regression_output_mae: 0.9351
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - classification_output_accuracy: 0.5027 - classification_output_loss: 0.7708 - loss: 2.5081 - regression_output_loss: 1.7373 - regression_output_mae: 1.0452 - val_classificatio

In [2]:
# ------------------------------
# 6. Evaluate the model
# ------------------------------
results = model.evaluate(
    X_reg, {"regression_output": y_reg, "classification_output": y_clf}, verbose=0
)
print("\nEvaluation results:")
print(f"Total Loss: {results[0]:.4f}")
print(f"Regression Loss (MSE): {results[1]:.4f}, MAE: {results[2]:.4f}")
print(f"Classification Loss (BCE): {results[3]:.4f}, Accuracy: {results[4]:.4f}")

# ------------------------------
# 7. Predict on new data
# ------------------------------
sample_input = X_reg[:5]   # take first 5 samples
reg_pred, clf_pred = model.predict(sample_input)

print("\nSample Inputs (first 5 rows):")
print(sample_input)

print("\nRegression Predictions:")
print(reg_pred.flatten())

print("\nClassification Predictions (probabilities):")
print(clf_pred.flatten())

print("\nClassification Predictions (class labels):")
print((clf_pred.flatten() > 0.5).astype(int))



Evaluation results:
Total Loss: 1.5038
Regression Loss (MSE): 0.9758, MAE: 0.5176
Classification Loss (BCE): 0.8000, Accuracy: 0.7958
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step

Sample Inputs (first 5 rows):
[[0.42341509 0.18913936 0.9136621  0.43569519 0.81426491]
 [0.95283545 0.09160411 0.59090621 0.96642365 0.47217696]
 [0.32597976 0.02413127 0.28263802 0.29197517 0.793393  ]
 [0.54372842 0.65346353 0.73212023 0.01959767 0.72947956]
 [0.37088331 0.71790835 0.762582   0.4947884  0.07308545]]

Regression Predictions:
[1.6639775 3.000011  1.2324712 2.9976082 2.5101933]

Classification Predictions (probabilities):
[0.331355   0.4783712  0.3100912  0.6566119  0.65425384]

Classification Predictions (class labels):
[0 0 0 1 1]
